This notebook is for testing x-clip

create model. head is frozen xclip vision model, then add on a few linear layers

In [28]:
# import lightning.pytorch as pl

# class delhi(pl.LightningModule):
#     def __init__(self,
#                  )

prepare data

In [29]:
import csv

data = {}

# Step 1: Open the CSV file
with open("/Users/jonathanlin/Documents/GitHub/research_transfer/datasets/Animal_Kingdom/action_recognition/annotation/val.csv", 'r') as csvfile:

    # Step 2: Create a CSV reader object
    csvreader = csv.reader(csvfile, delimiter=' ')

    # Step 3: Iterate through the rows
    for i, row in enumerate(csvreader):
        # Step 4: Process each row
        if i == 0:
            continue
        video_name = row[0]
        labels = row[4].split(",")
        data[video_name] = labels

video_dir = "/Users/jonathanlin/Documents/GitHub/research_transfer/datasets/Animal_Kingdom/action_recognition/dataset/video"

create a model that is the x-clip model as the head, frozen, and the 

In [30]:
import av
import torch
import numpy as np

from transformers import AutoProcessor, XCLIPVisionModel
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
# file_path = hf_hub_download(
#     repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
# )
container = av.open("/Users/jonathanlin/Documents/GitHub/research_transfer/datasets/Animal_Kingdom/action_recognition/dataset/video/AACKOHGA.mp4")

# sample 16 frames
indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = XCLIPVisionModel.from_pretrained("microsoft/xclip-base-patch32")

pixel_values = processor(videos=list(video), return_tensors="pt").pixel_values

batch_size, num_frames, num_channels, height, width = pixel_values.shape
pixel_values = pixel_values.reshape(-1, num_channels, height, width)

outputs = model(pixel_values)
last_hidden_state = outputs.last_hidden_state

No accelerated colorspace conversion found from yuv420p to rgb24.
Some weights of the model checkpoint at microsoft/xclip-base-patch32 were not used when initializing XCLIPVisionModel: ['prompts_generator.decoder.1.norm1.bias', 'text_model.encoder.layers.4.mlp.fc1.bias', 'text_model.encoder.layers.2.mlp.fc2.weight', 'text_model.encoder.layers.9.self_attn.v_proj.bias', 'text_model.encoder.layers.3.self_attn.q_proj.bias', 'text_model.encoder.layers.7.mlp.fc2.weight', 'prompts_generator.decoder.1.norm3.bias', 'text_model.encoder.layers.11.layer_norm2.bias', 'text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder.layers.4.self_attn.out_proj.bias', 'text_model.encoder.layers.11.mlp.fc1.bias', 'text_model.encoder.layers.4.mlp.fc2.weight', 'text_model.encoder.layers.0.layer_norm1.bias', 'mit.encoder.layers.0.layer_norm1.bias', 'text_model.encoder.layers.2.self_attn.out_proj.bias', 'text_model.encoder.layers.5.self_attn.v_proj.weight', 'mit.encoder.layers.0.mlp.fc2.weight', 

In [31]:
print(last_hidden_state.shape)

torch.Size([8, 50, 768])
